In [1]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline

sys.path.append(os.path.dirname(os.path.abspath(os.getcwd())) + "/..")
if os.getcwd().endswith("Topic2"):
    os.chdir("..")
    print(f"Changed to {os.getcwd()}")

from codebase import load_excel, load_odin_as_ml_dataset, plot_confusion_matrix, make_ml_dataset, run_binary_regression, transport_modes, trip_motives, province_codes, transport_mode_col

Changed to c:\Users\flori\OneDrive - TU Eindhoven\Master TUe 2024-2025\Design-of-AI-driven-business-operation


In [ ]:
years = [2019, 2020, 2021, 2022, 2023]
dfs = []
for year in years:
    odin_excel_path = os.path.join(os.getcwd(), "data", "OdiN 2019-2023", f"OdiN {year}", f"ODiN{year}_Databestand.xlsx")
    if year in [2019, 2020]:
        odin_excel_path = odin_excel_path.replace("Databestand", "Databestand_v2.0")
    df = load_excel(odin_excel_path)
    dfs.append(df)

df_all = pd.concat(dfs, ignore_index=True)

c:\ProgramData\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Loaded file from Excel and saved to CSV
Loaded file from Excel and saved to CSV


In [ ]:
len(df), len(df["OPID"].unique())

In [2]:
import pandas as pd, textwrap, re, unicodedata

FILE = "ODiN2022_Codeboek_v1.0.xlsx"    # put the workbook in the same folder
DROP = {"OP","OPID","Steekproef","Mode","Corop","BuurtAdam",
        "KLeeft","Jaar","Maand","Week","Dag","Weekdag","Feestdag"}
MAX_CODES = 30                          # show ≤ 10 labels

df = pd.read_excel(FILE,dtype=str).fillna("")
vars_ = {}
cur = None
for _,r in df.iterrows():
    v = r["Variabele_naam_ODiN_2022"].strip()
    if v:
        cur=v; vars_[v]={"level":r["Niveau"].strip(),
                         "desc":r["Variabele_label_ODiN_2022"].strip(),
                         "codes":[]}
    elif cur and r["Code_ODiN_2022"].strip():
        vars_[cur]["codes"].append(f'{r["Code_ODiN_2022"].strip()} = {r["Code_label_ODiN_2022"].strip()}')

# --- crude classifier (tweak if needed) ---------------------------------
num,bin,ord,cat = [],[],[],[]
for v,m in vars_.items():
    if v in DROP: continue
    codes=[c.split("=")[0].strip() for c in m["codes"]]
    if len(codes)>0 and all(re.fullmatch(r"\d+",c) for c in codes):
        if len(set(codes))==2 and set(codes)<= {"0","1"}:
            bin.append(v)
        elif codes==sorted(codes) and len(codes)<=15:
            ord.append(v)
        else:
            num.append(v)
    else:
        cat.append(v)
# ------------------------------------------------------------------------

def show(lst):
    out=[]
    for v in lst:
        c = vars_[v]["codes"][:MAX_CODES]
        lbl = ", ".join(c)+("…" if len(vars_[v]["codes"])>MAX_CODES else "")
        out.append(f'    "{v}", # {vars_[v]["level"]} – {vars_[v]["desc"]} ({lbl})')
    return out

print("# numerical_cols =",len(num)); print("[\n"+ "\n".join(show(num))+"\n]\n")
print("# binary_cols =",len(bin));    print("[\n"+ "\n".join(show(bin))+"\n]\n")
print("# ordinal_cols =",len(ord));   print("[\n"+ "\n".join(show(ord))+"\n]\n")
print("# categorical_cols =",len(cat));print("[\n"+ "\n".join(show(cat))+"\n]")


# numerical_cols = 30
[
    "Prov", # P – Provincie woongemeente (1 = Groningen, 2 = Fryslân, 3 = Drenthe, 4 = Overijssel, 5 = Flevoland, 6 = Gelderland, 7 = Utrecht, 8 = Noord-Holland, 9 = Zuid-Holland, 10 = Zeeland, 11 = Noord-Brabant, 12 = Limburg)
    "MRA", # P – Metropoolregio Amsterdam (1 = Stadsdeel Centrum, 2 = Stadsdeel Noord, 3 = Stadsdeel West (incl. Westpoort), 4 = Stadsdeel Nieuw-West, 5 = Stadsdeel Zuid, 6 = Stadsdeel Oost, 7 = Stadsdeel Zuidoost, 8 = Basisregio Waterland, 9 = Basisregio Zaanstreek, 10 = Basisregio IJmond, 11 = Basisregio Zuid-Kennemerland, 12 = Basisregio Meerlanden, 13 = Basisregio Amstelland, 14 = Basisregio Gooi en Vechtstreek, 15 = Basisregio Almere + Zeewolde, 16 = Basisregio Lelystad + Dronten, 17 = Noord-Holland Noord – Regio Alkmaar, 18 = Noord-Holland Noord – West-Friesland, 19 = Noord-Holland Noord – Kop van Noord-Holland, 20 = Urk & Noordoostpolder, 21 = Rest Nederland)
    "MRDH", # P – Metropoolregio Rotterdam Den Haag (1 = Den Haag Centrum

c:\ProgramData\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [ ]:
# Here just in case
year = 2022
odin_excel_path = os.path.join(os.getcwd(), "data", "OdiN 2019-2023", f"OdiN {year}", f"ODiN{year}_Databestand.csv")
df = load_excel(odin_excel_path)
df = df[df["Verpl"] == 1]
len(df)

In [ ]:
len(df_all) / df_all["VertPC"].unique().shape[0]

In [ ]:
# Create a histogram for the number of trips for each transport mode and translate to 
df[transport_mode_col].value_counts().plot(kind="bar", title="Number of trips per transport mode")

In [ ]:
for val, name in transport_modes.items():
    print(f"Number of {name}: ({len(df[df[transport_mode_col] == val])})")

In [ ]:
print(transport_modes)

In [ ]:
print(trip_motives)

In [ ]:
print(province_codes)

In [ ]:
df.columns

In [ ]:
max_dist=600 # max distance in 100s of meters
motives=None 
vehicles_to_include = [1, 2, 3, 7, 8, 9, ]
for transport_mode in vehicles_to_include:
    run_binary_regression(
        df,
        test_size=0.2,
        transport_modes_predict=[transport_mode, ],
        motives=motives,
        max_dist=max_dist,
        savename=f"graphics/classification_results/binary_regression/{motives}/{transport_modes[transport_mode].replace(r'/', ' or ')}_{max_dist}",
    )

In [ ]:
combined_modes = [[7, 8, 9]]
for tm in combined_modes:
    transport_mode_str = ", ".join([transport_modes[tm_] for tm_ in tm])
    run_binary_regression(
        df,
        test_size=0.2,
        transport_modes_predict=tm,
        motives=motives,
        max_dist=max_dist,
        savename=f"graphics/classification_results/binary_regression/{motives}/{transport_mode_str.replace(r'/', ' or ')}_{max_dist}",
    )

Now a classifier that classifies what mode of transport was used

In [ ]:
print(transport_modes)

In [ ]:
from codebase import run_multiclass_classification, distance_col, origin_col, province_codes, level_mapping_suffix, age_col, transport_mode_col, motive_col, urbanness_col


df_multiclass = df[df[transport_mode_col] != df[transport_mode_col].max()]
motives=None
origins=None#[7, 8, ]
location_level=2
categorical_features=[origin_col]
numerical_features=[distance_col, urbanness_col, motive_col]
origin_str = ', '.join(province_codes[code] for code in origins) if origins is not None else "NL"
motive_str = ', '.join(trip_motives[motive] for motive in motives) if motives is not None else "all motives"

_ = run_multiclass_classification(
    df_multiclass,
    test_size=0.2,
    destinations=None,
    origins=origins,
    location_level=location_level,
    categorical_features=categorical_features,
    numerical_features=numerical_features,
    motives=motives,
    plot=True,
    savename=f"graphics/classification_results/multiclass/{level_mapping_suffix[location_level]}/{origins}/{motives}/{categorical_features + numerical_features}",
    plot_title=f"Confusion Matrix for {origin_str} and {motive_str}",
)

In [ ]:
from codebase import urbanness_col, motive_col

age_range = (0, np.inf)

df_less_vehicles = df_multiclass[df_multiclass[transport_mode_col].isin(vehicles_to_include)]
df_less_vehicles = df_less_vehicles[(df_less_vehicles[age_col] < age_range[1]) & (df_less_vehicles[age_col] > age_range[0])]

motives=None
origins=None#[7, 8, ]
location_level=2
categorical_features=[origin_col]
numerical_features=[distance_col, urbanness_col, motive_col]
origin_str = ', '.join(province_codes[code] for code in origins) if origins is not None else "NL"

_ = run_multiclass_classification(
    df_less_vehicles,
    test_size=0.2,
    destinations=None,
    origins=origins,
    location_level=location_level,
    categorical_features=categorical_features,
    numerical_features=numerical_features,
    motives=motives,
    plot=True,
    savename=f"graphics/classification_results/multiclass/{level_mapping_suffix[location_level]}/{origins}/less_vehicles/age_{age_range[0]}_{age_range[1]}/{categorical_features + numerical_features}",
    plot_title=f"Confusion Matrix for {origin_str} by {', '.join(transport_modes[tm] for tm in vehicles_to_include)}",
)

In [ ]:
# Include a subset af transport modes to compare the bike to

run_binary_regression(
        df_less_vehicles[df_less_vehicles[transport_mode_col].isin([1, 8,])],
        test_size=0.2,
        transport_modes_predict=[8, ],
        motives=motives,
        max_dist=max_dist,
        savename=f"graphics/classification_results/binary_regression/car_foot_and_bike/{transport_modes[transport_mode].replace(r'/', ' or ')}_{max_dist}",
    )